---
title: Core
subtitle: Core data structures and algorithms
---


In [12]:
# | default_exp core
# | export
from pydantic import ConfigDict
from pydantic.dataclasses import dataclass
from datetime import datetime

## Variable

In [15]:
# | export
@dataclass(config=ConfigDict(extra='allow'))
class Variable:
    name: str = None
    description: str = None
    unit: str = None
    timerange: list[datetime] = None
    
@dataclass
class Variables:
    variables: dict[str, Variable] = None
    def add_variable(self, variable: Variable):
        self.variables[variable.name] = variable

In [17]:
# | export
import polars as pl
from functools import cached_property
from abc import abstractmethod

In [18]:
# | export
@dataclass
class SVariable(Variable):
    # similar to `speasy`
    provider: str = "cda"
    dataset: str = None
    parameter: str = None
    product: str = None
    """product name should be unique"""

    @abstractmethod
    def to_polars(self) -> pl.LazyFrame:
        ...

    def preview(self):
        return self.to_polars().head().collect()

    @abstractmethod
    @cached_property
    def data(self):
        """Retrieve the data if not already done."""
        ...

    def dump(self, path: str):
        """Dump the configuration to a file."""
        import yaml

        with open(path, "w") as f:
            yy = yaml.load(
                self.model_dump_json(exclude_defaults=True), Loader=yaml.FullLoader
            )
            yaml.dump(yy, f)

    @classmethod
    def load_from_file(cls, path: str):
        """Load the configuration from a file."""
        import yaml

        with open(path, "r") as f:
            yy = yaml.load(f, Loader=yaml.FullLoader)
            return cls(**yy)
        
class SVariables:

    products: list[str] = None
    parameters: list[str] = None

    _data: list[Variable] = None

## Dataset

In [21]:
# | export
@dataclass
class Dataset(Variables):
    name: str = None

## Instrument and Mission

In [ ]:
# | export
@dataclass
class Instrument:
    name: str
    datasets: dict[str, Dataset] = None

    def add_dataset(self, dataset: Dataset):
        self.datasets[dataset.name] = dataset


@dataclass
class Mission:
    name: str
    """Name of the mission"""
    instruments: dict[str, Instrument] = None
    datasets: dict[str, Dataset] = None

    def add_instrument(self, instrument: Instrument):
        self.instruments[instrument.name] = instrument

In [13]:
# | hide
import nbdev

nbdev.nbdev_export()